In [238]:
import pandas as pd 
import os
from utils.explanations import minmax_norm
from utils.utils import read_dataset
from utils.constants import CAM_LAYERS
import tensorflow.keras as keras
import tensorflow as tf
import numpy as np
from utils.explanations import calculate_cam_attributions
import matplotlib.pyplot as plt

import tensorflow_addons as tfa
#from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.utils import CustomObjectScope
def load_model(root_dir, archive_name, dataset_name, classifier, data_source): 
    with CustomObjectScope({'InstanceNormalization':tfa.layers.InstanceNormalization()}):
        model_path = f'{root_dir}/results/{archive_name}/{dataset_name}/' \
                                                + f'{classifier.split("_")[0]}/{classifier}/{data_source}/' \
                                                + f'best_model.hdf5'
        model =keras.models.load_model(model_path ,compile=False)
        return model



root_dir ='G:/Meine Ablage/master thesis/code/xai-tsc'
archive_name = 'ucr'
classifier = 'fcn_1.0'
dataset_name = 'Beef'
data_source = 'original'

data = read_dataset(root_dir, 'ucr', dataset_name,  data_source, 1)[dataset_name]

base_model = load_model(root_dir, archive_name, dataset_name, classifier, data_source)

#G:\Meine Ablage\master thesis\code\xai-tsc\results\ucr\GunPoint\fcn\fcn_mt_dense_0.5d

classifier = 'fcn_mt_dense_0.75'
data_source = 'fcn_raw'
dataset_name = 'GunPoint'
data_source = 'fcn_raw_mse'

sigmoid_model =  load_model(root_dir, archive_name, dataset_name, classifier, data_source)


In [244]:
print(data[0][0].shape)

(470,)


TypeError: 'Functional' object is not subscriptable

In [101]:
from utils.constants import CAM_LAYERS
from utils.explanations import get_layer_index


bact = CAM_LAYERS[classifier.split("_")[0]]["gap_layer"]
bact = get_layer_index(base_model, bact)

sact = CAM_LAYERS[classifier.split("_")[0]]["gap_layer"]
sact = get_layer_index(sigmoid_model, sact)

bnew_input_layer = base_model.inputs
bnew_output_layer = [base_model.layers[bact].output]
bew_feed_forward = keras.backend.function(bnew_input_layer, bnew_output_layer)


sew_input_layer = sigmoid_model.inputs
sew_output_layer = [sigmoid_model.layers[sact].output]
sew_feed_forward = keras.backend.function(sew_input_layer, sew_output_layer)


ts = data[0]

[bconv_out] = bew_feed_forward([ts])
[sconv_out] = sew_feed_forward([ts])

In [ ]:
mean_corr = 0
highest_filter_corr = 0
for i in range(128):
    #print(np.corrcoef(bconv_out[0][i],sconv_out[0][i]))
    mean_corr += np.corrcoef(bconv_out[0][i],sconv_out[0][i])
    for j in range(128):
        #print(np.corrcoef(bconv_out[0][i],sconv_out[0][i]))
        if np.corrcoef(bconv_out[0][i],sconv_out[0][i])[0,1] > highest_filter_corr:
            highest_filter_corr = np.corrcoef(bconv_out[0][i],sconv_out[0][i])[0,1] 


print(mean_corr / 128, highest_filter_corr)

In [164]:
def visualize_filter(model):
    import tensorflow.keras as keras
    root_dir ='G:/Meine Ablage/master thesis/code/xai-tsc'
    archive_name = 'ucr'
    classifier = 'fcn_1.0'
    dataset_name = 'GunPoint'
    data_source = 'original'
    datasets_dict = read_dataset(root_dir, 'ucr', dataset_name,  data_source, 1)
    x_train = datasets_dict[dataset_name][0]
    y_train = datasets_dict[dataset_name][1]

    print(x_train.shape)

    # filters
    filters = model.layers[1].get_weights()[0]

    new_input_layer = model.inputs
    new_output_layer = [model.layers[1].output]

    new_feed_forward = keras.backend.function(new_input_layer, new_output_layer)

    classes = np.unique(y_train)

    colors = [(255 / 255, 160 / 255, 14 / 255), (181 / 255, 87 / 255, 181 / 255)]
    colors_conv = [(210 / 255, 0 / 255, 0 / 255), (27 / 255, 32 / 255, 101 / 255)]

    idx = 10
    idx_filter = 10

    filter = filters[:, 0, idx_filter]

    print("Filter",filter)

    plt.figure()
    plt.plot(filter + 0.5, color='gray', label='filter')
    arr = []
    for c in classes:
        print(np.where(y_train == c))
        c_x_train = x_train[np.where(y_train == c)[0]]
        print(c_x_train.shape)
        convolved_filter_1 = new_feed_forward([c_x_train])[0]
        arr.append(convolved_filter_1)
        idx_c = int(c) - 1
        for f in range(128):
        #plt.plot(c_x_train[idx], color=colors[idx_c], label='class' + str(idx_c) + '-raw')
            plt.plot(convolved_filter_1[idx, :, f], color=colors_conv[idx_c], label='class' + str(idx_c) + '-conv')
        #plt.legend()

    root_dir = "./"
    plt.savefig(root_dir + 'convolution-' + dataset_name + classifier + '.pdf')

    return arr

In [165]:
arr0 = visualize_filter(base_model)

(50, 150)
Filter [ 0.04862259 -0.0022816   0.04238752  0.01836677 -0.01380017  0.0750989
 -0.08303187 -0.18231207]
(array([ 2,  3,  9, 10, 11, 12, 13, 15, 18, 20, 21, 22, 24, 26, 27, 29, 30,
       33, 35, 41, 42, 43, 46, 48], dtype=int64), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0], dtype=int64))
(24, 150)
(array([ 0,  1,  4,  5,  6,  7,  8, 14, 16, 17, 19, 23, 25, 28, 31, 32, 34,
       36, 37, 38, 39, 40, 44, 45, 47, 49], dtype=int64), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0], dtype=int64))
(26, 150)


In [250]:
def visualize_filter(model, yval=data[1]):
    import tensorflow.keras as keras
    root_dir ='G:/Meine Ablage/master thesis/code/xai-tsc'
    archive_name = 'ucr'
    classifier = 'fcn_mt_dense_0.0'
    data_source = 'fcn_minmax'
    dataset_name = 'Beef'
    datasets_dict = read_dataset(root_dir, 'ucr', dataset_name,  data_source, 470)
    x_train = datasets_dict[dataset_name][0]
    y_train = yval

    print(x_train.shape)

    # filters
    filters = model.layers[1].get_weights()[0]

    new_input_layer = model.inputs
    new_output_layer = [model.layers[1].output]

    new_feed_forward = keras.backend.function(new_input_layer, new_output_layer)
    
    classes = np.unique(y_train)

    colors = [(255 / 255, 160 / 255, 14 / 255), (181 / 255, 87 / 255, 181 / 255)]
    colors_conv = [(210 / 255, 0 / 255, 0 / 255), (27 / 255, 32 / 255, 101 / 255)]

    idx = 4
    idx_filter = 5

    filter = filters[:, 0, idx_filter]
    print("FILTER", filter)
    plt.figure()
    plt.plot(filter + 0.5, color='gray', label='filter')

    arr = []
    for c in classes:
        print(np.where(y_train == c))
        c_x_train = x_train[np.where(y_train == c)[0]]
        print(c_x_train.shape)
        convolved_filter_1 = new_feed_forward([c_x_train])[0]

        arr.append(convolved_filter_1)

        idx_c = int(c) - 1
        for f in range(10):
        #plt.plot(c_x_train[idx], color=colors[idx_c], label='class' + str(idx_c) + '-raw')
            plt.plot(convolved_filter_1[idx, :, f], color=colors_conv[idx_c], label='class' + str(idx_c) + '-conv')
        #plt.plot(c_x_train[idx], color=colors[idx_c], label='class' + str(idx_c) + '-raw')
        
        #plt.plot(convolved_filter_1[idx, :, idx_filter], color=colors_conv[idx_c], label='class' + str(idx_c) + '-conv')
        #plt.legend()

    root_dir = "./"
    plt.savefig(root_dir + 'convolution-' + dataset_name + classifier+ "t"+ '.pdf')

    return arr

In [246]:
arr1 = visualize_filter(sigmoid_model)

(30, 470)
FILTER [-0.09121471 -0.05040509 -0.06130533 -0.01264581  0.10143563  0.06047529
  0.0205708   0.12233275]
(array([0, 1, 2, 3, 4, 5], dtype=int64), array([0, 0, 0, 0, 0, 0], dtype=int64))
(6, 470)


ValueError: Input 0 of layer "model_80" is incompatible with the layer: expected shape=(None, 150, 1), found shape=(6, 470)

In [105]:
for i in arr1: 
    print(i.shape)

highest_match = 0
ix = None
ij = None
for i in range(128): 
    for j in range(128):
        if highest_match < (np.corrcoef(arr0[0][0][i],arr1[0][0][j]))[0,1]: 
            highest_match = (np.corrcoef(arr0[0][0][i],arr1[0][0][j]))[0,1]
            ix = i 
            ij = j 
print(highest_match)

(24, 150, 128)
(26, 150, 128)
0.4839307032093153


In [99]:
ix, ij

(53, 54)

In [234]:
classifier = 'fcn_mt_dense_0.0'
data_source = 'fcn_minmax_mse'
dataset_name = 'Beef'



dense_model_t =  load_model(root_dir, archive_name, dataset_name, classifier, data_source)

In [251]:
arr2 = visualize_filter(dense_model_t)

(30, 470)
FILTER [-0.13518423 -0.08784258 -0.09750187 -0.04375723  0.08166257  0.04844217
  0.01836685  0.11919203]
(array([0, 1, 2, 3, 4, 5], dtype=int64), array([0, 0, 0, 0, 0, 0], dtype=int64))
(6, 470)
(array([ 6,  7,  8,  9, 10, 11], dtype=int64), array([0, 0, 0, 0, 0, 0], dtype=int64))
(6, 470)
(array([12, 13, 14, 15, 16, 17], dtype=int64), array([0, 0, 0, 0, 0, 0], dtype=int64))
(6, 470)


IndexError: list index out of range

In [113]:
highest_match = 0
ix = None
ij = None
meanf = 0
for i in range(128): 
    for j in range(128):
        if highest_match < (np.corrcoef(arr0[0][0][i],arr2[0][0][j]))[0,1]: 
            highest_match = (np.corrcoef(arr0[0][0][i],arr2[0][0][j]))[0,1]
            ix = i 
            ij = j 
print(highest_match,ix,ij)

0.7286358455286505 52 52


In [116]:
arr0[0][0][52].mean(),arr2[0][0][52].mean()

In [230]:
arr1 = visualize_filter(dense_model_t)

(30, 790)
FILTER [-0.13518423 -0.08784258 -0.09750187 -0.04375723  0.08166257  0.04844217
  0.01836685  0.11919203]
(array([ 2,  3,  9, 10, 11, 12, 13, 15, 18, 20, 21, 22, 24, 26, 27, 29, 30,
       33, 35, 41, 42, 43, 46, 48], dtype=int64), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0], dtype=int64))


IndexError: index 30 is out of bounds for axis 0 with size 30

In [224]:
import numpy as np
from scipy.spatial.distance import cosine
from scipy.stats import pearsonr
def compare_filters_similarity(model1, model2, layer_index):
    filters1 = model1.layers[layer_index].get_weights()[0]
    filters2 = model2.layers[layer_index].get_weights()[0]
    print(filters1.shape)
    num_filters = filters1.shape[2]
    total_similarity = 0.0

    # Calculate similarity between filters
    for i in range(num_filters):
        max_similarity = 0.0
        for j in range(num_filters):
            filter1_i = filters1[:, :, i].flatten()
            filter2_j = filters2[:, :, j].flatten()
            #print(pearsonr(filter1_i, filter2_j)[0])
            similarity =  pearsonr(filter1_i, filter2_j)[0] #1 -cosine(filter1_i, filter2_j)
            if similarity > max_similarity:
                max_similarity = similarity
        total_similarity += max_similarity

    average_similarity = total_similarity / num_filters

    return average_similarity

compare_filters_similarity(base_model, dense_model_t, 1)

(8, 1, 128)


0.8971725667984465